# Final Project

In [1]:
!pip install --upgrade seaborn --q

In [2]:
from datetime import date

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from timeit import default_timer as Timer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

RANDOM_STATE = 12345

## Description
The telecom operator Interconnect would like to be able to forecast their churn of clients. If it's discovered that a user is planning to leave, they will be offered promotional codes and special plan options. Interconnect's marketing team has collected some of their clientele's personal data, including information about their plans and contracts.

### Data Description

The data consists of files obtained from different sources:

- `contract.csv` — contract information
- `personal.csv` — the client's personal data
- `internet.csv` — information about Internet services
- `phone.csv` — information about telephone services

In each file, the column `customerID` contains a unique code assigned to each client.

The contract information is valid as of February 1, 2020.

<a id='load_data'></a>
## Initial Data Review

### Download the data

In [4]:
from sys import platform

path = '/datasets/final_provider/'
if platform == 'darwin':
    path = '/Users/stainer/Desktop/Jupyter/Data_Sets/final_provider/'

In [5]:
try:
    contract_data = pd.read_csv(path + 'contract.csv')
    internet_data = pd.read_csv(path + 'internet.csv')
    personal_data = pd.read_csv(path + 'personal.csv')
    phone_data = pd.read_csv(path + 'phone.csv')
    data_dict = {'contract':contract_data, 'internet':internet_data, 'personal':personal_data, 'phone':phone_data}
    for data_set in data_dict:
        df = data_dict[data_set]
        df.columns = df.columns.str.lower()
except:
    print('Wrong file path')

### First glance on the data

In [6]:
for data_set in data_dict:
    df = data_dict[data_set]
    display(data_set, df.shape, df.head())

'contract'

(7043, 8)

,customerid,begindate,enddate,type,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


'internet'

(5517, 8)

...

(7043, 5)

,customerid,gender,seniorcitizen,partner,dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


'phone'

(6361, 2)

,customerid,multiplelines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


<a id='1'></a>
#### Question
__What should be our target?__<br>
Client wants to know whether a clint gonna churn next month.<br>
EndDate field suits that purpose. Existing date - will churn, 'No' - will not.<br>
For the purpose of model training we should create a new field, assigning True for churn, and false for the opposite.

In [7]:
contract_data['target'] = (contract_data['enddate'] != 'No')
contract_data['target'].value_counts()

False    5174
True     1869
Name: target, dtype: int64

<a id='2'></a>
#### Question
Classes are unbalanced.
__Should we use upsampling, downsampling or some other method of class balancing?__<br>
We probably will test those options during the model training. However, having limited data, downsampling seems problematic

### Inspect data shapes

In [8]:
for data_set in data_dict:
    df = data_dict[data_set]
    df.columns = df.columns.str.lower()
    print(f"{data_set:<10} shape: {df.shape}")

contract   shape: (7043, 9)
internet   shape: (5517, 8)
personal   shape: (7043, 5)
phone      shape: (6361, 2)


<a id='3'></a>
#### Question. 
Internet and phone data contains less entries than contract and personal data. __How should we take care of those missing values?<br>__
This is happening probably due to some customers using only phone or only internet services.<br>
During the stage, where we merge all the data into one table, missing values should probably be filled with a placeholder meaning user do not use this particular service.

### Check data types and inspect for missing values

In [9]:
for data_set in data_dict:
    df = data_dict[data_set]
    display(data_set, df.dtypes)
    print("No missing values were found" if df.isna().sum().sum() == 0 else "Found missing values")

'contract'

customerid           object
begindate            object
enddate              object
type                 object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
target                 bool
dtype: object

No missing values were found


'internet'

customerid          object
internetservice     object
onlinesecurity      object
onlinebackup        object
deviceprotection    object
techsupport         object
streamingtv         object
streamingmovies     object
dtype: object

...

customerid       object
gender           object
seniorcitizen     int64
partner          object
dependents       object
dtype: object

No missing values were found


'phone'

customerid       object
multiplelines    object
dtype: object

No missing values were found


In [10]:
def total_charges_to_int(total_charges):
    """Convert object values of total charges to float
    
    :param row: Total charges value as object
    :return: Value converted to float or 0
    
    >>> total_charges_to_int('123.4')
    123.4
    """
    try:
        return float(total_charges)
    except:
        return 0.0

In [11]:
charges = contract_data['totalcharges'].apply(total_charges_to_int)

<a id='4'></a>
We found no missng values inside provided data.<br>
However data types in some instances do not fit.
#### Questions:
- Begin date and end date are objects, not datetime. In addition enddate field has object values 'No'. __How we should process them?__
    - Option one: Leave them be, as we are probably gonna use them to extract contract length, to use as a feature, and will not use further. And target was already extracted from the enddate field.
    - Option two: Replace 'No' values with actual date(2020-02-01)
- Total charges field should be of type float, but we have empty values. 
    - __Should we ommit those records or set them to 0?__ Having contract length of 0 those records could distort our data.
- As for other values, __how should we process missing values that we will get while merging data?__
    - Such values should be replaced with values like 'None'. ex. internet service field would have three possible values: 'fiber', 'DSL', 'None' - for customers who do not use this service.
    - Additionally we should double check for customers, not using any service, to be shure.

<a id='5'></a>
#### Question.
__What features should we use?__<br>
- Which services provided the company customer uses and billing details.
    - We will have to assign values for services customer do not use and categorize those fields
- Length of contract
- Monthly payment
    - Total payment is ommited, as redundant. As it is most likely a product of monthly payment nultiplyed by contract length

- Date fields are used to extract target and features and will not be used further
- Customer Id is just a unique identificator

## Questions summary
- What should we use as a target? [here](#1)
- How should we process class imballance? [here](#2)
- As we merge data, we will have missing values. How shoud we process them? [here](#3)
- How should we process data types? [here](#4)
- What features will we use for model training? [here](#5)

## Working Plan:
- [Download and inspect the data](#load_data)
- [Merge tables, fill missing values and fix data types](#preprocess)
- [Extract target and features](#feature_extraction)
- Model training
    - [Encode and scale features. Split data into train and test sets](#train_test_split)
    - [Model training](#models)
- [Performance evaluation](#evaluation)

<a id='preprocess'></a>
## Preprocessing

### Merge tables

In [12]:
full_data = contract_data.merge(personal_data, how='outer').merge(phone_data, how='outer').merge(internet_data, how='outer')

print(full_data.shape)
# Check for missing values
full_data.isna().sum()

(7043, 21)


customerid             0
begindate              0
enddate                0
type                   0
paperlessbilling       0
paymentmethod          0
monthlycharges         0
totalcharges           0
target                 0
gender                 0
seniorcitizen          0
partner                0
dependents             0
multiplelines        682
internetservice     1526
onlinesecurity      1526
onlinebackup        1526
deviceprotection    1526
techsupport         1526
streamingtv         1526
streamingmovies     1526
dtype: int64

### Check if there are customers that do not use any services

In [13]:
phone_services = phone_data.columns.to_list()
phone_services.remove('customerid')
internet_services = internet_data.columns.to_list()
internet_services.remove('customerid')

# look for customers with missing values in all fields related to provided services
if (pd.isnull(full_data[phone_services+internet_services]).sum(axis=1) > 7).sum() != 0:
    print("There are customers that do not use any services")
else:
    print("There are no 'dead' customers")

There are no 'dead' customers


### Replace missing values

Missing values in columns with multiple choice will be replaced with "None"<br>
While missing values in columns with binary choice(online security, online backup etc) will be replaced with "No"

In [14]:
# multiple lines possible values: Yes - multiple lines, No - single line, None - no lines
# internet service possible values: DSL, Fiber, None - do not use this service
multiple_choice = ['multiplelines', 'internetservice']
# fields below have two options Yes - service in use, No - service not in use
binary_choice = ['onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies']

full_data[multiple_choice] = full_data[multiple_choice].fillna('None')
full_data[binary_choice] = full_data[binary_choice].fillna('No')

if full_data.isna().sum().sum() == 0:
    print("Missing values were successfully filled")
else:
    print("Some missing values still remain")

Missing values were successfully filled


In [15]:
cat_features =  full_data.columns.to_list()
num_features = ['monthlycharges', 'duration']

# convert object fields to category
for column in ['customerid', 'begindate', 'enddate', 'monthlycharges', 'totalcharges', 'target']: cat_features.remove(column)
full_data[cat_features] = full_data[cat_features].astype('category')

### Extract contract length from dates

In [16]:
def calc_turn(row):
    """ Calculate contract length till turner, or actual date in days
    
    :param row: Data Frame row
    :return: End date minus begin date
    
    >>> calc_turn(['2020-01-01', '2020-01-10'])
    9
    """
    begin = pd.to_datetime(row['begindate'])
    try:
        end = pd.to_datetime(row['enddate'])
    except:
        # end = date.today()
        end = pd.to_datetime('2020-02-01')
    return (end - begin).days

In [17]:
full_data['duration'] = full_data.apply(calc_turn, axis=1)

### Extract features and target

In [18]:
features = full_data[num_features + cat_features]
target = full_data['target'] * 1

### Summary:
- We've merged provided data
- Filled missing values caused by merging tables that didn't have data in customers who do not use certain services
- Changed data types
- Used dates of beginning and ending the contract to calculate contract length
- Extracted features and target to separate data sets

<a id='train_test_split'></a>
## Model training
### Encode categorical feartures and split data into train and test sets

In [19]:
# dictionary to save model performance
results = {}

In [20]:
# encode categorical features
ohe_fearures = pd.get_dummies(features[cat_features], drop_first=True)
# join with numerical features
ohe_features = ohe_fearures.join(features[num_features])
# split and check for class 1 share in each set
train_features, test_features, train_target, test_target = train_test_split(ohe_features, target, test_size=0.25, random_state = RANDOM_STATE)
print(f"Train set have {train_target.sum()/train_target.shape[0]:.2%} True targets")
print(f"Test set have {test_target.sum()/test_target.shape[0]:.2%} True targets")

Train set have 26.41% True targets
Test set have 26.92% True targets


### Scale numerical

In [21]:
scaler = MinMaxScaler()
scaler.fit(train_features[num_features])
train_features[num_features] = scaler.transform(train_features[num_features])
test_features[num_features] = scaler.transform(test_features[num_features])

<ipython-input-21-e7bd5775e40b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features[num_features] = scaler.transform(train_features[num_features])
<ipython-input-21-e7bd5775e40b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_features[num_features] = scaler.transform(test_features[num_features])


<a id='models'></a>
### Train models

In [22]:
def evaluate_model(model, model_name, train_features, train_target, test_features, test_target, results):
    """ Train and test model. Save results and train/prediction times to dictionary
    
    :param model: Model for training
    :param model_name: Name for dictionary entrie
    :param train_features: Training features
    :param train_traget: Training target
    :param test_features: Test features
    :param test_target: Test target
    :param results: Results dictionary
    """
    # create dictionary entrie for a model
    results[model_name] = {}
    # train and record training time
    begin = Timer()
    model.fit(train_features, train_target)
    end = Timer()
    results[model_name]['train time'] = end - begin
    
    predictions_train = model.predict_proba(train_features)[:, 1]
    # make predictions for a test set and record prediction time
    begin = Timer()
    predictions_test = model.predict_proba(test_features)[:, 1]
    end = Timer()
    results[model_name]['predict time'] = end - begin
    # save roc-auc score for train and test predictions
    results[model_name]['roc-auc train'] = roc_auc_score(train_target, predictions_train)
    results[model_name]['roc-auc test'] = roc_auc_score(test_target, predictions_test)
    results[model_name]['f1 train'] = f1_score(train_target, model.predict(train_features))
    results[model_name]['f1 test'] = f1_score(test_target, model.predict(test_features))

Now we will declare model, pass it to this function with needed data, then repeat until satisfied :)<br>

[skip to results](#evaluation)

#### Dummy Classifier

In [23]:
dummy = DummyClassifier()

evaluate_model(dummy, 'Dummy', train_features, train_target, test_features, test_target, results)

#### Logistic Regression

In [24]:
lr = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')

evaluate_model(lr, 'Logistic Regression', train_features, train_target, test_features, test_target, results)

#### Logistic Regressoion (Balanced)

In [25]:
lrb = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='liblinear')

evaluate_model(lrb, 'Logistic Regression (Balanced)', train_features, train_target, test_features, test_target, results)

#### Logistic Regression (Upsampled)

In [26]:
def upsample_set(features, target):
    """ Upsamples less represented class, shuffles and returns features and target
    
    :param features: Features of unbalanced data set
    :param target: Target of unbalanced data set
    :return: Balanced features and target
    """
    # Split by class
    features_true = features[target==True]
    features_false = features[target==False]
    
    target_true = target[target==True]
    target_false = target[target==False]
    # Repeat underrepresented class and join with the second class
    if len(target_false) > len(target_true):
        ratio = int(len(target_false) / len(target_true))
        features_upsampled = pd.concat([features_false] + [features_true] * ratio)
        target_upsampled = pd.concat([target_false] + [target_true] * ratio)
    else:
        ratio = int(len(target_true) / len(target_false))
        features_upsampled = pd.concat([features_true] + [features_false] * ratio)
        target_upsampled = pd.concat([target_true] + [target_false] * ratio)
    # Shuffle and return
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=RANDOM_STATE)
    return features_upsampled, target_upsampled


In [27]:
up_features, up_target = upsample_set(train_features, train_target)

lru = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')

evaluate_model(lru, 'Logistic Regression (Upsampled)', up_features, up_target, test_features, test_target, results)


#### Decision Tree

In [28]:
parameters = {'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
dtc = GridSearchCV(DecisionTreeClassifier(random_state=RANDOM_STATE), parameters, cv=4, scoring = 'roc_auc')

evaluate_model(dtc, 'Decision Tree Classifier', train_features, train_target, test_features, test_target, results)

#### Decision Tree (Balanced)

In [29]:
parameters = {'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
dtcb = GridSearchCV(DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced'), parameters, cv=4, scoring = 'roc_auc')

evaluate_model(dtcb, 'Decision Tree Classifier (Balanced)', train_features, train_target, test_features, test_target, results)

#### Decision Tree (Upsampled)

In [30]:
up_features, up_target = upsample_set(train_features, train_target)

parameters = {'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
dtcu = GridSearchCV(DecisionTreeClassifier(random_state=RANDOM_STATE), parameters, cv=4, scoring = 'roc_auc')

evaluate_model(dtcu, 'Decision Tree Classifier (Upsampled)', up_features, up_target, test_features, test_target, results)

#### Random Forest

In [31]:
parameters = {'n_estimators': [100], 'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
rf = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), parameters, cv=5, scoring = 'roc_auc')

evaluate_model(rf, 'Random Forest Classifier', train_features, train_target, test_features, test_target, results)

#### Random Forest (Balanced)

In [32]:
parameters = {'n_estimators': [100], 'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
rfb = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced'), parameters, cv=5, scoring = 'roc_auc')

evaluate_model(rfb, 'Random Forest Classifier (Balanced)', train_features, train_target, test_features, test_target, results)

#### Random Forest (Upsampled)

In [33]:
up_features, up_target = upsample_set(train_features, train_target)

parameters = {'n_estimators': [100], 'max_depth': range(3, 20, 2), 'min_samples_split' : range(2, 20, 2)}
rfu = GridSearchCV(RandomForestClassifier(random_state=RANDOM_STATE), parameters, cv=5, scoring = 'roc_auc')

evaluate_model(rfu, 'Random Forest Classifier (Upsampled)', up_features, up_target, test_features, test_target, results)

#### Catboost

In [34]:
# parameters = {'learning_rate' : np.arange(0.05, 0.2, 0.05), 'verbose': [0]}
# cat = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss'), parameters, cv=3, scoring = 'roc_auc')
cat = CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss', verbose=0)
evaluate_model(cat, 'CatBoost Classifier', train_features, train_target, test_features, test_target, results)


#### Catboost (Balanced)

In [35]:
classes = np.unique(train_target)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_target)

# parameters = {'learning_rate' : np.arange(0.05, 0.2, 0.05), 'verbose': [0]}
# catb = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss', class_weights=weights), parameters, cv=3, scoring = 'roc_auc')
catb = CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss', class_weights=weights, verbose=0)
evaluate_model(catb, 'CatBoost Classifier (Balanced)', train_features, train_target, test_features, test_target, results)


#### CatBoost (Upsampled)

In [36]:
up_features, up_target = upsample_set(train_features, train_target)

# parameters = {'learning_rate' : np.arange(0.05, 0.2, 0.05), 'verbose': [0]}
# catb = GridSearchCV(CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss', class_weights=weights), parameters, cv=3, scoring = 'roc_auc')
catu = CatBoostClassifier(random_state=RANDOM_STATE, loss_function='Logloss', verbose=0)
evaluate_model(catu, 'CatBoost Classifier (Upsampled)', up_features, up_target, test_features, test_target, results)


<a id='evaluation'></a>
### Performance Evaluation

In [37]:
# print(f"{'model':^40}|{'train roc-auc':>15}|{'test roc-auc':>15}|{'train time':>15}|{'predict time':>15}|")
# print('-' * 105)
# for model in results:
#     entrie = results[model]
#     try:
#         print(f"{model:^40}|{entrie['roc-auc train']:>15.4f}|{entrie['roc-auc test']:>15.4f}|{entrie['train time']:>15.4f}|{entrie['predict time']:>15.4f}|")
#     except:
#         pass

In [38]:
table = pd.DataFrame(results).transpose()
# table.sort_values(by='roc-auc test', ascending=False)
table

,train time,predict time,roc-auc train,roc-auc test,f1 train,f1 test
Dummy,0.000963,0.000175,0.500000,0.500000,0.000000,0.000000
Logistic Regression,0.049841,0.002004,0.848404,0.838684,0.593493,0.590542
Logistic Regression (Balanced),0.024877,0.002278,0.848286,0.839121,0.630503,0.629660
Logistic Regression (Upsampled),0.029883,0.001754,0.848422,0.839002,0.724741,0.628781
Decision Tree Classifier,4.355032,0.001389,0.858697,0.839191,0.639425,0.625821
Decision Tree Classifier (Balanced),4.630971,0.001138,0.886608,0.844780,0.672803,0.627622
Decision Tree Classifier (Upsampled),5.266313,0.001346,0.970840,0.765837,0.878189,0.589796
Random Forest Classifier,157.719275,0.034237,0.911962,0.854969,0.658364,0.591961
Random Forest Classifier (Balanced),190.623289,0.033212,0.912105,0.853357,0.713206,0.646801
Random Forest Classifier (Upsampled),172.449492,0.040629,0.999979,0.835719,0.996064,0.612766


### Recap
- We've encoded categorical features
- Split data into train and test data sets
- Scaled numerical features
- Trained couple of models, recording their performance
    - Dummy Classifier
    - Logistic Regression. With optional class balancing.
    - Decision Tree Classifier. With hyperparameter tuning and optional class balancing.
    - Random Forest Classifier. With hyperparameter tuning and optional class balancing.
    - CatBoostClassifier. With optional class balancing. Tuning learning rate took a long time while affecting performance very slightly, so ommited it. 

### Performance Evaluation
- CatboostClassifier delivered best results while maintaining adequate training and prediction times. 
- Using built-in balancing feature helped improving Logistic Regression and Decision Tree performance, while worsening the score of Random Fores and  CatBoost.
- Upsampling worked even worse with Decision Tree and Random Forest, but improved slightly CatBoost score.

Overall CatBoost did a great job and is recomended for use with class upsampling. Performance is best around the table, while timings are still reasonable, even if it would be necessary to perform this process once a month with larger pool of data.